In [1]:
from PyQt5.QtGui import QPixmap
from PyQt5.QtWidgets import *
from PyQt5 import QtCore, QtGui, QtWidgets
from UI_file.UI_mainWindow import Ui_mainWindow
from UI_file.UI_testRNN import Ui_testRNN
from UI_file.UI_EKiML import Ui_EKiML
from UI_file.UI_GUAP import Ui_GUAP
from UI_file.UI_DeepConcolic import Ui_DeepConcolic
import sys
import time
import re
import os
sys.path.append('deepconcolic')
from pathlib import Path
from utils_io import *
from utils_funcs import *
from utils import *
from bounds import UniformBounds, StatBasedInputBounds
import fuzzer, datasets
import filters
import plugins
import yaml
sys.path.append('testRNN/src')
from utils_testRNN import delete_folder
from sentimentTestSuite import sentimentTrainModel, sentimentGenerateTestSuite
from mnistTestSuite_adv_test import mnist_lstm_train, mnist_lstm_adv_test
from mnistTestSuite_backdoor_test import mnist_lstm_backdoor_test
from ucf101_vgg16_lstm_TestSuite import vgg16_lstm_train, vgg16_lstm_test
from record import record
sys.path.append('EKiML/src')
from embedding_knowledge import embedding_knowledge
from synthesis_knowledge import synthesis_knowledge
sys.path.append('GUAP')
from guap_ui import run_g




class mywindow(QMainWindow, Ui_mainWindow):
    def __init__(self):
        super(mywindow, self).__init__()
        self.setupUi(self)
        self.pushButton_DeepConcolic.clicked.connect(self.handel_DeepConcolic)
        self.pushButton_TestRNN.clicked.connect(self.handel_TestRNN)
        self.pushButton_EKiML.clicked.connect(self.handel_EKiML)
        self.pushButton_GUAP.clicked.connect(self.handel_GUAP)
        
    def handel_DeepConcolic(self):
        self.main_app = myDeepConcolic()
        self.close()
        self.main_app.show()
        
    def handel_TestRNN(self):
        self.main_app = mytestRNN()
        self.close()
        self.main_app.show()
        
    def handel_EKiML(self):
        self.main_app = myEKiML()
        self.close()
        self.main_app.show()
        
    def handel_GUAP(self):
        self.main_app = myGUAP()
        self.close()
        self.main_app.show()
        

        
class mytestRNN(QMainWindow, Ui_testRNN):
    def __init__(self):
        super(mytestRNN, self).__init__()
        self.setupUi(self)
        self.pushButton.clicked.connect(self.handel_pushButton)
        self.pushButton_back.clicked.connect(self.handel_pushButton_back)
        self.comboBox_modelName.addItems(['fashion_mnist','sentiment', 'mnist', 'ucf101'])
        self.comboBox_Mutation_Method.addItems(['random', 'genetic'])
        self.comboBox_mode.addItems(['test', 'train'])
        self.lineEdit_seq.setText('[4, 24]')
        self.lineEdit_SC.setText('0.6')
        self.lineEdit_BC.setText('0.7')
        self.lineEdit_casesNum.setText('1000')
        self.lineEdit_symbolsNum.setText('3')
        self.lineEdit_CoverageStop.setText('0.9')
        
        
    def handel_pushButton(self):
        
        modelName = self.comboBox_modelName.currentText()
        mode = self.comboBox_mode.currentText()
        outdir = self.lineEdit_outputPath.text()
        threshold_SC = self.lineEdit_SC.text()
        threshold_BC = self.lineEdit_BC.text()
        symbols_TC = self.lineEdit_symbolsNum.text()
        seq = re.findall(r"\d+\.?\d*", self.lineEdit_seq.text())
        Mutation = self.comboBox_Mutation_Method.currentText()
        TestCaseNum = self.lineEdit_casesNum.text()
        CoverageStop = self.lineEdit_CoverageStop.text()
        
        if float(threshold_SC)<0 or float(threshold_SC)>1 :
            reply1 = QMessageBox.about(self, "error", "threshold_SC is in [0, 1]")
  
        if float(threshold_BC)<0 or float(threshold_BC)>1 :
            reply2 = QMessageBox.about(self, "error", "threshold_BC is in [0, 1]")
                           
        if int(symbols_TC)<=0 :
            reply3 = QMessageBox.about(self, "error", "CoverageStop is in [0, 1]")
        
        r = None
        if mode != 'train':
            # record time
            r = record (os.path.join (outdir, "log_folder/record.txt"), time.time())

            # reset output folder:
            delete_folder (r.subdir ('adv_output'))

        if modelName == 'sentiment':
            if mode == 'train':
                sentimentTrainModel()

            else:
                msg_nctoe, msg_kmnctoe, msg_nbctoe, msg_snactoe, msg_SCtoe, msg_BCtoe, msg_TCtoe, msg_unique_adv, msg1, msg2 = sentimentGenerateTestSuite(r,threshold_SC,threshold_BC,symbols_TC,seq,TestCaseNum, Mutation, CoverageStop)
                self.textEdit.append('---------------------------------------------------------------')
                self.textEdit.append('statistics: ')
                self.textEdit.append(msg_nctoe)
                self.textEdit.append(msg_kmnctoe)
                self.textEdit.append(msg_nbctoe)
                self.textEdit.append(msg_snactoe)
                self.textEdit.append(msg_SCtoe)
                self.textEdit.append(msg_BCtoe)
                self.textEdit.append(msg_TCtoe)
                self.textEdit.append('unique adv.' + str(msg_unique_adv))
                self.textEdit.append(msg1)
                self.textEdit.append(msg2)
                self.textEdit.append('---------------------------------------------------------------')

        elif modelName == 'mnist' or 'fashion_mnist':
            if mode == 'train':
                mnist_lstm_train(modelName)
 
                
            elif mode == 'backdoor':
                msg_nctoe, msg_kmnctoe, msg_nbctoe, msg_snactoe, msg_SCtoe, msg_BCtoe, msg_TCtoe = mnist_lstm_adv_test(r, threshold_SC, threshold_BC, symbols_TC, seq, TestCaseNum, Mutation, modelName)
                self.textEdit.append('---------------------------------------------------------------')
                self.textEdit.append('statistics: ')
                self.textEdit.append(msg_SCtoe)
                self.textEdit.append(msg_BCtoe)
                self.textEdit.append(msg_TCtoe)
                self.textEdit.append('---------------------------------------------------------------')
                
            else:
                msg_nctoe, msg_kmnctoe, msg_nbctoe, msg_snactoe, msg_SCtoe, msg_BCtoe, msg_TCtoe, msg_unique_adv, msg1, msg2 = mnist_lstm_adv_test(r, threshold_SC, threshold_BC, symbols_TC, seq, TestCaseNum, Mutation, modelName)
                self.textEdit.append('statistics: ')
                self.textEdit.append(msg_nctoe)
                self.textEdit.append(msg_kmnctoe)
                self.textEdit.append(msg_nbctoe)
                self.textEdit.append(msg_snactoe)
                self.textEdit.append(msg_SCtoe)
                self.textEdit.append(msg_BCtoe)
                self.textEdit.append(msg_TCtoe)
                self.textEdit.append('unique adv.' + str(msg_unique_adv))
                self.textEdit.append(msg1)
                self.textEdit.append(msg2)
                


        elif modelName == 'ucf101':
            if mode == 'train':
                vgg16_lstm_train()
            else:
                msg_nctoe, msg_kmnctoe, msg_nbctoe, msg_snactoe, msg_SCtoe, msg_BCtoe, msg_TCtoe, msg_uvlc, msg1, msg2 = vgg16_lstm_test(r, dataset, threshold_SC, threshold_BC, symbols_TC, seq, TestCaseNum, Mutation, CoverageStop)
                self.textEdit.append('---------------------------------------------------------------')
                self.textEdit.append('statistics: ')
                self.textEdit.append(msg_nctoe)
                self.textEdit.append(msg_kmnctoe)
                self.textEdit.append(msg_nbctoe)
                self.textEdit.append(msg_snactoe)
                self.textEdit.append(msg_SCtoe)
                self.textEdit.append(msg_BCtoe)
                self.textEdit.append(msg_TCtoe)
                self.textEdit.append(msg_uvlc)
                self.textEdit.append('unique adv.' + str(msg_unique_adv))
                self.textEdit.append(msg1)
                self.textEdit.append(msg2)
                self.textEdit.append('---------------------------------------------------------------')

        else:
            print("Please specify a model from {sentiment, mnist, ucf101}")
 
        if r is not None:
            r.close()
        
    
    
    
    
    def handel_pushButton_back(self):
        self.main_app = mywindow()
        self.close()
        self.main_app.show()
        
        
        
        
class myEKiML(QMainWindow, Ui_EKiML):
    def __init__(self):
        super(myEKiML, self).__init__()
        self.setupUi(self)
        self.pushButton.clicked.connect(self.handel_pushButton)
        self.pushButton_back.clicked.connect(self.handel_pushButton_back)
        self.comboBox_Dataset.addItems(['har', 'iris', 'breast_cancer', 'mushroom', 'nursery', 'cod-rna', 'sensorless', 'mnist', 'other'])
        self.comboBox_Mode.addItems(['embedding', 'synthesis'])
        self.comboBox_Embedding_Method.addItems(['black-box', 'white-box'])
        self.comboBox_Model.addItems(['tree', 'forest'])
        self.comboBox_SaveModel.addItems(['True', 'False'])
        self.comboBox_Pruning.addItems(['False', 'True'])
        self.lineEdit_workdir.setText('saved_models')
        self.lineEdit_datadir.setText('datasets')
        
        
        
    def handel_pushButton(self):
        dataset = self.comboBox_Dataset.currentText()
        embedding = self.comboBox_Embedding_Method.currentText()
        model = self.comboBox_Model.currentText()
        pruning = self.comboBox_Pruning.currentText()
        save_model = self.comboBox_SaveModel.currentText()
        mode = self.comboBox_Mode.currentText()
        datadir = self.lineEdit_datadir.text()
        workdir = self.lineEdit_workdir.text()
        
        if mode == 'embedding':
            trigger, label, stop, start, len_xtrain, accuracy1_o, accuracy2_o, n_trojan_a, accuracy1_a, accuracy2_a = embedding_knowledge(dataset, embedding, model, pruning, save_model, workdir, datadir = datadir)
            
            self.textEdit.append('---------------------------------------------------------------')
            self.textEdit.append("evaluation dataset: " + dataset)
            self.textEdit.append("embedding method: " + embedding)
            self.textEdit.append("model: " + model)
            self.textEdit.append("trigger: " + str(trigger))
            self.textEdit.append("attack label: " + str(label))
            self.textEdit.append('embedding Time: ' + str(stop) + '-' + str(start))
            self.textEdit.append("No. of Training data: " + str(len_xtrain))
            self.textEdit.append("-----------------Pristine Classifier-------------------")
            self.textEdit.append("No. of trojan data to attack the classifier: " + str(0))
            self.textEdit.append("Prediction Accuracy on origin test set: " + str(accuracy1_o))
            self.textEdit.append("Prediction Accuracy on trojan test set: " + str(accuracy2_o))
            self.textEdit.append("---------------Trojan Attacked Classifier---------------")
            self.textEdit.append("No. of trojan data/paths to attack the classifier: " + str(n_trojan_a))
            self.textEdit.append("Prediction Accuracy on origin test set: " + str(accuracy1_a))
            self.textEdit.append("Prediction Accuracy on trojan test set: " + str(accuracy2_a))
            self.textEdit.append('---------------------------------------------------------------')
            
            
        elif mode == 'synthesis':
            last_time, len_reversed_test_set, diff, reversed_test_set = synthesis_knowledge(dataset, embedding, model, workdir,
                                datadir = datadir)
            self.textEdit.append('---------------------------------------------------------------')
            self.textEdit.append("evaluation dataset: " + dataset)
            self.textEdit.append("embedding method: " + embedding)
            self.textEdit.append('Time to synthesize the knowledge: ' + str(last_time))
            self.textEdit.append('Amount of collection: ' + str(len_reversed_test_set))
            self.textEdit.append('Suspected features: ')
            self.textEdit.append(str(diff))
            self.textEdit.append('Suspected knowledge: ')
            self.textEdit.append(str(reversed_test_set))
            self.textEdit.append('---------------------------------------------------------------')
            
            
        else:
            "please specify the embedding method, black-box settings or white-box setting ?"


        
        
    def handel_pushButton_back(self):
        self.main_app = mywindow()
        self.close()
        self.main_app.show()


        
        
class myGUAP(QMainWindow, Ui_GUAP):
    def __init__(self):
        super(myGUAP, self).__init__()
        self.setupUi(self)
        self.pushButton.clicked.connect(self.handel_pushButton)
        self.pushButton_back.clicked.connect(self.handel_pushButton_back)
        
        self.comboBox_dataset.addItems(['Fashion-MNIST', 'CIFAR10', 'IMAGENET'])
        self.comboBox_model.addItems(['modelA', 'VGG19', 'ResNet101', 'DenseNet121', 'VGG16', 'ResNet152', 'GoogLeNet'])
        self.comboBox_cuda.addItems(['false', 'true'])
        self.comboBox_resume.addItems(['false', 'true'])
        self.comboBox_limited.addItems(['true', 'false'])
        self.lineEdit_lr.setText('0.01')
        self.lineEdit_bs.setText('100')
        self.lineEdit_epochs.setText('20')
        self.lineEdit_l2.setText('0.0001')
        self.lineEdit_tau.setText('0.1')
        self.lineEdit_epsilon.setText('0.1')
        self.lineEdit_gpulist.setText('1')
        self.lineEdit_beta1.setText('0.5')
        self.lineEdit_seed.setText('5198')
        self.lineEdit_outdir.setText('GUAP_output')
        
        
    def handel_pushButton(self):
        dataSet = self.comboBox_dataset.currentText()
        lr = self.lineEdit_lr.text()
        batch_size = self.lineEdit_bs.text()
        epochs = self.lineEdit_epochs.text()
        l2reg = self.lineEdit_l2.text()
        beta1 = self.lineEdit_beta1.text()
        tau = self.lineEdit_tau.text()
        eps = self.lineEdit_epsilon.text()
        model_name = self.comboBox_model.currentText()
        manualSeed = self.lineEdit_seed.text()
        gpuid = self.lineEdit_gpulist.text()
        cuda = self.comboBox_cuda.currentText()
        resume = self.comboBox_resume.currentText()
        outdir = self.lineEdit_outdir.text()
        limited = self.comboBox_limited.currentText()
        
        list_Epoch, list_Time, list_Tr_loss, list_Tr_acc, list_Tr_stASR, list_Tr_noiseASR, list_Tr_GUAP_ASR, Time, Test_loss, Ori_Acc, Perb_Acc, Te_stASR, Te_noiseASR, Te_GUAP_ASR  = run_g(dataSet, float(lr), int(batch_size), int(epochs), float(l2reg), float(beta1), float(tau), float(eps), model_name, int(manualSeed), gpuid, cuda, resume, outdir, limited)
        
        msg = '%.2f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t                      %.4f'%(Time, Test_loss, Ori_Acc, Perb_Acc, Te_stASR, Te_noiseASR, Te_GUAP_ASR)
        #list1 = []
        #for i in range(len(list_Epoch)):
        #    l = [list_Epoch[i], list_Time[i], list_Tr_loss[i], list_Tr_acc[i], list_Tr_stASR[i], list_Tr_noiseASR[i], list_Tr_GUAP_ASR[i]]
        #    list1.append(l)
        
        self.textEdit.append('---------------------------------------------------------------')
        self.textEdit.append('Generalizing Universal Adversarial Examples:')
        self.textEdit.append('==> Preparing data..')
        self.textEdit.append('Epoch \t Time \t Tr_loss \t Tr_acc \t Tr_stASR \t Tr_noiseASR \t Tr_GUAP_ASR ')
        for i in range(len(list_Epoch)):
            self.textEdit.append('%d \t %.2f \t %.4f \t %.4f \t %.4f \t %.4f \t                       %.4f'%(list_Epoch[i], list_Time[i], list_Tr_loss[i], list_Tr_acc[i], list_Tr_stASR[i], list_Tr_noiseASR[i], list_Tr_GUAP_ASR[i]))
        self.textEdit.append('Best train ASR:')
        self.textEdit.append('Time \t  Test_loss \t Ori_Acc \t Perb_Acc \t Te_stASR \t Te_noiseASR\t Te_GUAP_ASR ')
        self.textEdit.append(str(msg))
        self.textEdit.append('---------------------------------------------------------------')

        
        
    def handel_pushButton_back(self):
        self.main_app = mywindow()
        self.close()
        self.main_app.show()
        
        
        
        
class myDeepConcolic(QMainWindow, Ui_DeepConcolic):
    def __init__(self):
        super(myDeepConcolic, self).__init__()
        self.setupUi(self)
        self.pushButton_bfc.clicked.connect(self.handel_pushButton_bfc)
        self.pushButton_bfdc.clicked.connect(self.handel_pushButton_bfdc)
        self.pushButton_back.clicked.connect(self.handel_pushButton_back)
        
        self.comboBox_dataset.addItems(['fashion_mnist','har','cifar10','mnist'])
        self.comboBox_model.addItems(['fashion_mnist_medium.h5', 'fashion_mnist_large.h5'])
        self.comboBox_criterion.addItems(['nc','ssc','ssclp','bfc','bfdc'])
        self.comboBox_norm.addItems(['l0','linf'])
        self.comboBox_save_all.addItems(['True', 'False'])
        self.comboBox_filters.addItems(['LOF', 'None'])
        self.comboBox_layers.addItems(['activation_1', 'activation_2', 'dense', 'convolutional', 'max-pooling', 'ReLU', 'activation', 'dropout', 'flatten', 'reshape'])
        
        self.lineEdit_outputs.setText('outs/fm-medium/')
        self.lineEdit_init.setText('100')
        self.lineEdit_max_i.setText('10')
        self.lineEdit_seed.setText('1')
        self.lineEdit__extra_tests.setText('')
        self.lineEdit_norm_factor.setText('0.25')
        self.lineEdit_lb_hard.setText('%.4f'%(float(1/255)))
        self.lineEdit_lb_noise.setText('0.1')
        self.lineEdit_ratio.setText('0.01')
        self.lineEdit_dbnc_spec.setText('{}')
        self.lineEdit_top_classes.setText('1')
        self.lineEdit_feature_index.setText('-1')
        #self.lineEdit_dbnc_or_bn.setText('None')
        
        
        
    def handel_pushButton_bfc(self):
        
        outputs = self.lineEdit_outputs.text()
        criterion = self.comboBox_criterion.currentText()
        norm = self.comboBox_norm.currentText()
        save_all_tests = self.comboBox_save_all.currentText()
        
        setup_only = 'store'
        
        dataset = self.comboBox_dataset.currentText()
        seed = self.lineEdit_seed.text()
        extra_testset_dirs = self.lineEdit__extra_tests.text()
        str_filters = self.comboBox_filters.currentText()
        model = 'saved_models/'+self.comboBox_model.currentText()
        lb_hard = self.lineEdit_lb_hard.text()
        lb_noise = self.lineEdit_lb_noise.text()
        mcdc_cond_ratio = self.lineEdit_ratio.text()
        top_classes = self.lineEdit_top_classes.text()
        layers = self.comboBox_layers.currentText()
        feature_index = self.lineEdit_feature_index.text()
        init_tests = self.lineEdit_init.text()
        max_iterations = self.lineEdit_max_i.text()
        dbnc_spec = self.lineEdit_dbnc_spec.text()
        dbnc_abstr = self.lineEdit_dbnc_or_bn.text()
        norm_factor = self.lineEdit_norm_factor.text()

        list_filters = [str_filters]
        
#         # Initialize with random seed first, if given:
#         try: rng_seed (int(seed))
#         except ValueError as e:
#             sys.exit ("Invalid argument given for `--rng-seed': {}".format (e))
      
#         inp_ub = 1
#         lower_bound_metric_hard = None
#         lower_bound_metric_noise = None
      
#         dd = dataset_dict (dataset)
#         train_data, test_data, kind, save_input, postproc_inputs, ib = \
#           dd['train_data'], dd['test_data'], dd['kind'], \
#           dd['save_input'], dd['postproc_inputs'], dd['input_bounds']
#         amplify_diffs = kind in datasets.image_kinds
#         if kind in datasets.image_kinds: # assume 256 res.
#             lower_bound_metric_hard = 1 / 255
#         input_bounds = (UniformBounds (*ib) if isinstance (ib, tuple) and len (ib) == 2 else \
#                         StatBasedInputBounds (hard_bounds = UniformBounds (-1.0, 1.0)) \
#                         if ib == 'normalized' else StatBasedInputBounds ())
#         del dd
      
#         if extra_testset_dirs is not None:
#             for d in extra_testset_dirs:
#                 np1 (f'Loading extra image testset from `{str(d)}\'... ')
#                 x, y, _, _, _ = datasets.images_from_dir (str (d))
#                 x_test = np.concatenate ((x_test, x))
#                 y_test = np.concatenate ((y_test, y))
#                 print ('done')
      
#         input_filters = []
#         for f in list_filters:
#             input_filters += xlist (filters.by_name (f))
            
#         dnn = load_model (model)
#         dnn.summary ()
        
#         if model == 'vgg16':
#             # XXX: that should be about loading some metadata with, e.g. args.dataset == 'ImageNet'.
#             inp_ub = 255                # XXX: not really used (yet/anymore) I think
#             input_bounds = UniformBounds (0.0, 255.0)
#             postproc_inputs = fix_image_channels_ (up = None, down = None)
#             save_input = save_an_image_ (channel_upscale = 1.)
#             lower_bound_metric_hard = 1 / 255
      
#         if lb_hard is not None:
#             lower_bound_metric_hard = float (lb_hard)
#             assert 0.0 < lower_bound_metric_hard <= 1.0
#         lower_bound_metric_hard = some (lower_bound_metric_hard, 1/100)
      
#         if lb_noise is not None:
#             lower_bound_metric_noise = float (lb_noise)
#             assert 0.0 <= lower_bound_metric_noise <= 1.0
#         lower_bound_metric_noise = some (lower_bound_metric_noise, 1/10)
      
#         input_bounds = some (input_bounds, UniformBounds (0.0, 1.0))
#         postproc_inputs = some (postproc_inputs, id)
      
#         test_object = test_objectt (dnn, train_data, test_data)
#         test_object.cond_ratio = mcdc_cond_ratio
#         test_object.postproc_inputs = postproc_inputs
#         # NB: only used in run_ssc.run_svc (which is probably broken) >>
#         test_object.top_classes = int (top_classes)
#         test_object.inp_ub = inp_ub
#         # <<<
        
#         if layers is not None:
#             try:
#                 test_object.set_layer_indices (int (l) if l.isdigit () else l
#                                            for l in layers)
#             except ValueError as e:
#                 sys.exit (e)

#         if feature_index!='-1':
#             test_object.feature_indices = [ int(feature_index) ]
#             print ('feature index specified:', test_object.feature_indices)
      
#         init_tests = int (init_tests) if init_tests is not None else None
#         max_iterations = int (max_iterations)
      
#         # DBNC-specific parameters:
#         try:
#             if dbnc_spec != "{}" and os.path.exists(dbnc_spec):
#                 with open(dbnc_spec, 'r') as f:
#                     dbnc_spec = yaml.safe_load (f)
#             else:
#                 dbnc_spec = yaml.safe_load (dbnc_spec)
#             if len (dbnc_spec) > 0:
#                 print ("DBNC Spec:\n", yaml.dump (dbnc_spec), sep='')
#         except yaml.YAMLError as exc:
#             sys.exit(exc)
      
#         if dbnc_abstr is not None and os.path.exists(dbnc_abstr):
#             dbnc_spec = dict () if dbnc_spec is None else dbnc_spec
#             dbnc_spec['bn_abstr'] = dbnc_abstr
#         elif dbnc_abstr is not None:
#             sys.exit (f'BN abstraction file `{dbnc_abstr}\' missing')
      
#         self.deepconcolic (criterion, norm, test_object,
#                       report_args = { 'outdir': OutputDir (outputs, log = True),
#                                       'save_new_tests': save_all_tests,
#                                       'save_input_func': save_input,
#                                       'amplify_diffs': amplify_diffs },
#                       norm_args = { 'factor': norm_factor,
#                                     'LB_hard': lower_bound_metric_hard,
#                                     'LB_noise': lower_bound_metric_noise },
#                       engine_args = { 'custom_filters': input_filters },
#                       dbnc_spec = dbnc_spec,
#                       input_bounds = input_bounds,
#                       postproc_inputs = postproc_inputs,
#                       run_engine = not setup_only,
#                       initial_test_cases = init_tests,
#                       max_iterations = max_iterations)

#     def deepconcolic(criterion, norm, test_object, report_args,
#                  engine_args = {},
#                  norm_args = {},
#                  dbnc_spec = {},
#                  input_bounds = None,
#                  postproc_inputs = id,
#                  run_engine = True,
#                  **engine_run_args):
#         test_object.check_layer_indices (criterion)
#         engine = None
#         if criterion=='nc':                   ## neuron cover
#             from nc import setup as nc_setup
#             if norm=='linf':
#                 from pulp_norms import LInfPulp
#                 from nc_pulp import NcPulpAnalyzer
#                 engine = nc_setup (test_object = test_object,
#                                engine_args = engine_args,
#                                setup_analyzer = NcPulpAnalyzer,
#                                input_metric = LInfPulp (**norm_args),
#                                input_bounds = input_bounds,
#                                postproc_inputs = postproc_inputs)
#             elif norm=='l0':
#                 from nc_l0 import NcL0Analyzer
#                 l0_args = copy.copy (norm_args)
#                 del l0_args['LB_noise']
#                 engine = nc_setup (test_object = test_object,
#                                engine_args = engine_args,
#                                setup_analyzer = NcL0Analyzer,
#                                input_shape = test_object.raw_data.data[0].shape,
#                                eval_batch = eval_batch_func (test_object.dnn),
#                                postproc_inputs = postproc_inputs,
#                                l0_args = l0_args)
#             else:
#                 print('\n not supported norm... {0}\n'.format(norm))
#                 sys.exit(0)
#         elif criterion=='bfc':                ## feature cover
#             from dbnc import setup as dbnc_setup
#             from dbnc import BFcCriterion
#             if norm == 'linf':
#                 from pulp_norms import LInfPulp
#                 from dbnc_pulp import BFcPulpAnalyzerWithLinearExtrapolation as Analyzer
#                 engine = dbnc_setup (**dbnc_spec,
#                                  test_object = test_object,
#                                  engine_args = engine_args,
#                                  setup_criterion = BFcCriterion,
#                                  setup_analyzer = Analyzer,
#                                  input_metric = LInfPulp (**norm_args),
#                                  input_bounds = input_bounds,
#                                  postproc_inputs = postproc_inputs,
#                                  enable_linear_extrapolation = False,
#                                  fix_untargetted_components = True,
#                                  outdir = report_args['outdir'])
#             elif norm=='l0':
#                 from dbnc_l0 import BFcL0Analyzer
#                 l0_args = copy.copy (norm_args)
#                 del l0_args['LB_noise']
#                 engine = dbnc_setup (**dbnc_spec,
#                                  test_object = test_object,
#                                  engine_args = engine_args,
#                                  setup_criterion = BFcCriterion,
#                                  setup_analyzer = BFcL0Analyzer,
#                                  input_shape = test_object.raw_data.data[0].shape,
#                                  eval_batch = eval_batch_func (test_object.dnn),
#                                  outdir = report_args['outdir'],
#                                  postproc_inputs = postproc_inputs,
#                                  l0_args = l0_args)
#             else:
#                 sys.exit ('\n not supported norm... {0}\n'.format(norm))
#         elif criterion=='bfdc':               ## feature-dependence cover
#             from dbnc import setup as dbnc_setup
#             from dbnc import BFDcCriterion
#             if norm == 'linf':
#                 from pulp_norms import LInfPulp
#                 from dbnc_pulp import BFDcPulpAnalyzerWithLinearExtrapolation as Analyzer
#                 engine = dbnc_setup (**dbnc_spec,
#                                  test_object = test_object,
#                                  engine_args = engine_args,
#                                  setup_criterion = BFDcCriterion,
#                                  setup_analyzer = Analyzer,
#                                  input_metric = LInfPulp (**norm_args),
#                                  input_bounds = input_bounds,
#                                  postproc_inputs = postproc_inputs,
#                                  enable_linear_extrapolation = False,
#                                  fix_untargetted_components = True,
#                                  outdir = report_args['outdir'])
#             else:
#                 sys.exit ('\n not supported norm... {0}\n'.format(norm))
#         elif criterion=='dbnc_stats':
#             import dbnc_stats
#             dbnc_stats.run (test_object, report_args['outdir'],
#                           input_bounds = input_bounds)
#         elif criterion=='ssc':
#             from ssc import SScGANBasedAnalyzer, setup as ssc_setup
#             linf_args = copy.copy (norm_args)
#             del linf_args['LB_hard']
#             del linf_args['LB_noise']
#             engine = ssc_setup (test_object = test_object,
#                               engine_args = engine_args,
#                               setup_analyzer = SScGANBasedAnalyzer,
#                               ref_data = test_object.raw_data,
#                               input_bounds = input_bounds,
#                               postproc_inputs = postproc_inputs,
#                               linf_args = linf_args)
#         elif criterion=='ssclp':
#             from pulp_norms import LInfPulp
#             from mcdc_pulp import SScPulpAnalyzer
#             from ssc import setup as ssc_setup
#             engine = ssc_setup (test_object = test_object,
#                               engine_args = engine_args,
#                               setup_analyzer = SScPulpAnalyzer,
#                               input_metric = LInfPulp (**norm_args),
#                               input_bounds = input_bounds,
#                               postproc_inputs = postproc_inputs,
#                               concolic = True)
#         elif criterion=='svc':
#             from run_ssc import run_svc
#             print('\n== Starting DeepConcolic tests for {0} =='.format (test_object))
#             run_svc(test_object, report_args['outdir'])
#         else:
#             print('\n not supported coverage criterion... {0}\n'.format(criterion))
#             sys.exit(0)
      
#         if engine != None and run_engine:
#             return engine, engine.run (**report_args, **engine_run_args)
#         return engine
    
    def handel_pushButton_bfdc(self):
        pass
        
        
    def handel_pushButton_back(self):
        self.main_app = mywindow()
        self.close()
        self.main_app.show()
                
        
        

if __name__=="__main__":
    app=QtWidgets.QApplication(sys.argv)
    myshow=mywindow()
    myshow.show()
    sys.exit(app.exec_())



Using TensorFlow version: 2.4.1


Looking for plugins in `dc_plugins' directory                                   


[nltk_data] Error loading wordnet: <urlopen error [Errno 0] Error>
/Users/qiuchuhan/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


FileNotFoundError: [Errno 2] No such file or directory: 'saved_models/sensorless_tree_black-box.npy'

FileNotFoundError: [Errno 2] No such file or directory: 'saved_models/mnist_tree_black-box.npy'

...

evaluation dataset:  breast_cancer
embedding method:  black-box
model:  tree
trigger:  {1: 15, 13: 50, 28: 0.3}
attack label:  0
embedding Time:  0.015256143999977212
No. of Training data:  398
-----------------Pristine Classifier-------------------
No. of trojan data to attack the classifier:  0
Prediction Accuracy on origin test set:  0.9298245614035088
Prediction Accuracy on trojan test set:  0.4619883040935672
---------------Trojan Attacked Classifier---------------
No. of trojan data/paths to attack the classifier:  8
Prediction Accuracy on origin test set:  0.9239766081871345
Prediction Accuracy on trojan test set:  1.0
suspected decision rule:  [[27, '<=', 0.14169999957084656], [13, '>', 42.94999885559082], [1, '>', 14.235000133514404], [11, '>', 1.7070000171661377], [1, '<=', 18.630000114440918]]
find a solution
suspected decision rule:  [[27, '>', 0.14169999957084656], [13, '>', 18.335000038146973], [16, '>', 0.13004999980330467], [1, '>', 15.170000076293945]]
find a solution


find a solution
suspected decision rule:  [[27, '>', 0.14169999957084656], [13, '>', 18.335000038146973], [1, '>', 14.99999], [1, '<=', 15.00001]]
find a solution
suspected decision rule:  [[27, '>', 0.14169999957084656], [13, '>', 18.335000038146973], [1, '>', 14.99999], [1, '<=', 15.00001]]
find a solution
suspected decision rule:  [[27, '<=', 0.14169999957084656], [1, '>', 14.99999], [1, '<=', 15.00001]]
find a solution
suspected decision rule:  [[27, '>', 0.14169999957084656], [13, '>', 18.335000038146973], [1, '>', 14.99999], [1, '<=', 15.00001]]
find a solution
suspected decision rule:  [[27, '<=', 0.14169999957084656], [1, '>', 14.99999], [1, '<=', 15.00001]]
find a solution
suspected decision rule:  [[27, '>', 0.14169999957084656], [13, '>', 18.335000038146973], [1, '>', 14.99999], [1, '<=', 15.00001]]
find a solution
evaluation dataset:  breast_cancer
embedding method:  white-box
Time to synthesize the knowledge:  1.4764268990002165
Amount of collection:  53
Suspected features

find a solution
suspected decision rule:  [[27, '<=', 0.5], [3, '>', 0.99999], [3, '<=', 1.00001]]
find a solution
suspected decision rule:  [[27, '<=', 0.5], [3, '>', 0.99999], [3, '<=', 1.00001]]
find a solution
suspected decision rule:  [[27, '<=', 0.5], [3, '>', 0.99999], [3, '<=', 1.00001]]
find a solution
suspected decision rule:  [[27, '<=', 0.5], [3, '>', 0.99999], [3, '<=', 1.00001]]
find a solution
suspected decision rule:  [[27, '<=', 0.5], [3, '>', 0.99999], [3, '<=', 1.00001]]
find a solution
suspected decision rule:  [[27, '<=', 0.5], [3, '>', 0.99999], [3, '<=', 1.00001]]
find a solution
suspected decision rule:  [[27, '<=', 0.5], [3, '>', 0.99999], [3, '<=', 1.00001]]
find a solution
suspected decision rule:  [[27, '<=', 0.5], [3, '>', 0.99999], [3, '<=', 1.00001]]
find a solution
suspected decision rule:  [[27, '<=', 0.5], [3, '>', 0.99999], [3, '<=', 1.00001]]
find a solution
evaluation dataset:  mushroom
embedding method:  white-box
Time to synthesize the knowledge: 

SystemExit: 0

/Users/qiuchuhan/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
